# Simple LangGraph Workflow
This notebook demonstrates how to create a minimal workflow with LangGraph and OpenAI.
## Create Client

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv

#add your OPENAI_API_KEY to .env
load_dotenv()
client = OpenAI()

## Test OpenAI Client
We send a simple prompt to verify the client is configured correctly.

In [12]:
from pprint import pprint
response = client.responses.create(
  model="gpt-4o-mini",
  input="Tell me a three sentence bedtime story about a unicorn."
)
message = response.output[0].content[0].text
pprint(message)

('In a hidden valley, a shy unicorn named Lila discovered a sparkling lake '
 'that shimmered under the moonlight. Each night, she would visit the lake and '
 'share her dreams with the twinkling stars, who listened and whispered back '
 'sweet secrets. One evening, the stars granted Lila a wish, and from that day '
 'on, her radiant glow lit up the night, bringing joy to all the creatures in '
 'her magical home.')


### Workflow
**Objective:** Route a query based on its sentiment to an appropriate response node.

In [40]:
def sentiment_analysis_node(state: GraphState) -> GraphState:
    query = state["query"]
    if any(word in query for word in ["happy", "joy"]):
        classification = "happy"
    else: 
        classification = "angry"
    return {"classification": classification}

def happy_response_node(state: GraphState) -> GraphState:
    return {"response": "Why so happy!"}

def angry_response_node(state: GraphState) -> GraphState:
    return {"response": "Why so angry!"}
        

In [41]:
def decide_node(state: GraphState) -> GraphState:
    if state["classification"] == "happy":
        return "happy_response"
    else:
        return "angry_response"

In [42]:
from typing import TypedDict, Optional
# this contains the information we need to pass between nodes
class GraphState(TypedDict):
    query: Optional[str] = None # initial 
    classification: Optional[str] = None # intermediate
    response: Optional[str] = None # final


In [43]:
from langgraph.graph import StateGraph, START,END


workflow = StateGraph(GraphState)


workflow.add_node("sentiment_analysis", sentiment_analysis_node)
workflow.add_node("happy_response", happy_response_node)
workflow.add_node("angry_response", angry_response_node)





workflow.add_edge(START, "sentiment_analysis")
workflow.add_conditional_edges("sentiment_analysis" , decide_node)

workflow.add_edge("happy_response",END)
workflow.add_edge("angry_response",END)

In [45]:
app = workflow.compile()

In [46]:
inputs = {"query": "I'm happy!!"}
result = app.invoke(inputs)
print(result)

{'query': "I'm happy!!", 'classification': 'happy', 'response': 'Why so happy!'}


In [47]:
inputs = {"query": "Life is gloomy"}
result = app.invoke(inputs)
print(result)

{'query': 'Life is gloomy', 'classification': 'angry', 'response': 'Why so angry!'}


## Improve the Workflow
Now that we have a working example, let's create a better sentiment analysis step.
We'll use gpt-4o-mini to classify the user's sentiment.

In [48]:
def gpt_sentiment_analysis_node(state: GraphState) -> GraphState:
    query = state["query"]
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Act as a sentiment analyzer and return happy or angry based on the query, don't return anything else"},
            {"role": "user", "content": query}
        ]
    )
    classification = response.choices[0].message.content
    return {"classification": classification}

In [49]:
from langgraph.graph import StateGraph, START,END


workflow = StateGraph(GraphState)


workflow.add_node("sentiment_analysis", gpt_sentiment_analysis_node)
workflow.add_node("happy_response", happy_response_node)
workflow.add_node("angry_response", angry_response_node)





workflow.add_edge(START, "sentiment_analysis")
workflow.add_conditional_edges("sentiment_analysis" , decide_node)

workflow.add_edge("happy_response",END)
workflow.add_edge("angry_response",END)

In [52]:
app = workflow.compile()

### Run the Improved Workflow
The gpt-4o-mini classifier gives us more accurate responses.

In [53]:
inputs = {"query": "Tommorow is a weekend"}
result = app.invoke(inputs)
print(result)

{'query': 'Tommorow is a weekend', 'classification': 'happy', 'response': 'Why so happy!'}


This concludes the short LangGraph demo.